In [ ]:
import requests
import pandas as pd
import math
import time

In [ ]:
#CareerOneStop/NLX
#First step is to gather job list; each job IDs will then be queried for specific job details/descriptions in the second step
#The maximum number of results is 500; script must iterate through pages to aggregate all the job IDs.
api_token = '2gaZEtABi5pxX1tFQeRgudXuWheouJKgYemjwUO58EH15WEKQJGw4NRXeDpbyH+r91fmkmv2ycO9uRH16yBIsQ=='    
user_id = 'cnzHhxx5yqwWiUl'
query_term = 'cybersecurity'
days = '0' #All days
start_page = 1
end_page = 500

url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
headers = {'Authorization': 'Bearer ' + api_token,
 'Content-Type': 'application/json'}
results = requests.get(url, headers=headers).json()



In [ ]:
#Calculate number of API calls to get all the results
jobs_total = int(results['Jobcount'])
total_pages = math.ceil(int(results['Jobcount'])/500)
print('Number of total jobs: ' + str(jobs_total) , '\nNumber of iterations/pages: ' + str(total_pages))

In [ ]:
job_list = []

#Loop through all pages of jobs
for i in range(total_pages):
    start_page += 500
    end_page += 500
    url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
    #Extract job IDs and append to master list
    results = requests.get(url, headers=headers).json()
    for i, value in enumerate(results['Jobs']):
        job_list.append(results['Jobs'][i]['JvId'])
    time.sleep(2.0)
        

In [ ]:
#De-deplicate the job ID list
dedupe_list = list(set(job_list))
print(str(len(dedupe_list)) + ' requests will be made to retrieve job details. Proceed carefully!' )

In [ ]:
#Retrieve job details using list of job IDs
###
#This will generate large number of calls to API!
###
jobs = pd.DataFrame()
for i, value in enumerate(dedupe_list):
    url_getjob = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{dedupe_list[i]}?isHtml=true&enableMetaData=true'
    results = requests.get(url_getjob, headers=headers).json()
    jobs = pd.concat([jobs, pd.DataFrame([[results['JobTitle'], results['Company'], results['Description']]])])
    time.sleep(1.0) #throttle the calls down to one second!

In [ ]:
jobs.reset_index(drop = True)

In [ ]:
jobs